# Data Driven Features

The EPC data contains several categorical variables with a lot of values. In order to find suitable features which will retain the most information, the data driven a eplored.

The first approach, termed data driven, uses statistical methods to reduce the number of variables. As the variables containing textual descriptions of the property have been created free-hand, many contain a large number of unique values. In some cases, only recorded for one property. The data driven approach uses a single level Chi-square Automatic Interaction Detector (CHAID) to group the levels within each categorical variable into a smaller number of groups. CHAID groups values with a similar response rate or in this context Energy Efficiency Rating (EER).

This script applies the groupings from the CHAID and bins the numerical fields 

In [58]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import json

In [117]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config-example.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_train_clean_fname = config['DEFAULT']['epc_train_clean_fname']
epc_test_clean_fname = config['DEFAULT']['epc_test_clean_fname']
epc_train_data_fname = config['DEFAULT']['epc_train_dd_fname']
epc_test_data_fname = config['DEFAULT']['epc_test_dd_fname']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']

In [118]:
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname) + epc_fname_suffix,header = 0,delimiter = ',',dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])
epc_test = pd.read_csv(os.path.join(processing_path,epc_test_clean_fname) + epc_fname_suffix,header = 0,delimiter = ',',dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])

C:\Users\KingRemy\AppData\Local\Temp\ipykernel_13636\1740213250.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname) + epc_fname_suffix,header = 0,delimiter = ',',dtype = dtype_dict,
C:\Users\KingRemy\AppData\Local\Temp\ipykernel_13636\1740213250.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_test = pd.read_csv(os.path.join(processing_path,epc_test_clean_fname) + epc_fname_suffix,header = 0,delimiter = ',',dtype = dtype_dict,


In [119]:
%store -r chaid_dict

## Combine bins of categorical variables into a smaller number of bins

### Built_form

In [120]:
# built_dict = dict.fromkeys(['Mid-Terrace','End-Terrace'],'terraced')
# built_dict1 = dict.fromkeys(['Semi-Detached','Detached'],'detached')
# built_dict.update(built_dict1)
# epc_train['built_form'] = epc_train['BUILT_FORM'].replace(built_dict)
# epc_test['built_form'] = epc_test['BUILT_FORM'].replace(built_dict)

In [121]:
chaid_dict['BUILT_FORM']['node1'].remove('<missing>')

In [122]:
built_dict = dict.fromkeys(chaid_dict['BUILT_FORM']['node1'], 'Other')
built_dict1 = dict.fromkeys(chaid_dict['BUILT_FORM']['node2'], 'Mid-Terrace')
built_dict2 = dict.fromkeys(chaid_dict['BUILT_FORM']['node3'], 'End-Terrace')
built_dict3 = dict.fromkeys(chaid_dict['BUILT_FORM']['node4'], 'Semi-Detached')
built_dict4 = dict.fromkeys(chaid_dict['BUILT_FORM']['node5'], 'Detached')
built_dict.update(built_dict1)
built_dict.update(built_dict2)
built_dict.update(built_dict3)
built_dict.update(built_dict4)
epc_train['built_form'] = epc_train['BUILT_FORM'].replace(built_dict)
epc_test['built_form'] = epc_test['BUILT_FORM'].replace(built_dict)

### Energy_tariff

In [123]:
# adding which is missing from chaid due to small volumes, and removing null from other groups
# chaid_dict['ENERGY_TARIFF']['node2'].append('off-peak 18 hour')
# chaid_dict['ENERGY_TARIFF']['node1'].remove('Unknown')
chaid_dict['ENERGY_TARIFF']['node1'].remove('<missing>')

In [124]:
# energy_dict = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node1'],'single/dual')
# energy_dict1 = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node2'],'off-peak')
# energy_dict['Unknown'] = np.nan 
# energy_dict.update(energy_dict1)
# epc_train['energy_tariff'] = epc_train['ENERGY_TARIFF'].replace(energy_dict)
# epc_test['energy_tariff'] = epc_test['ENERGY_TARIFF'].replace(energy_dict)

In [125]:
energy_dict = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node1'], 'Off-Peak')
energy_dict1 = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node2'], 'Standard Tariff')
energy_dict2 = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node3'], 'Single')
energy_dict3 = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node4'], 'Dual')
energy_dict4 = dict.fromkeys(chaid_dict['ENERGY_TARIFF']['node5'], 'I dont know')
energy_dict.update(energy_dict1)
energy_dict.update(energy_dict2)
energy_dict.update(energy_dict3)
energy_dict.update(energy_dict4)
epc_train['energy_tariff'] = epc_train['ENERGY_TARIFF'].replace(energy_dict)
epc_test['energy_tariff'] = epc_test['ENERGY_TARIFF'].replace(energy_dict)

### Floor description

In [126]:
chaid_dict['FLOOR_DESCRIPTION']['node1'].remove('<missing>')

In [127]:
floor_desc_dict = dict.fromkeys(chaid_dict['FLOOR_DESCRIPTION']['node1'],'Insulated concrete 1990 above')
floor_desc_dict1 = dict.fromkeys(chaid_dict['FLOOR_DESCRIPTION']['node2'],'Insulated Timber 1990 above')
floor_desc_dict2 = dict.fromkeys(chaid_dict['FLOOR_DESCRIPTION']['node3'],'Timber un-insulated floor pre 1990')
floor_desc_dict3 = dict.fromkeys(chaid_dict['FLOOR_DESCRIPTION']['node4'],'Concrete uninsulated floor pre 1990')
floor_desc_dict4 = dict.fromkeys(chaid_dict['FLOOR_DESCRIPTION']['node5'],'I dont know')
floor_desc_dict.update(floor_desc_dict1)
floor_desc_dict.update(floor_desc_dict2)
floor_desc_dict.update(floor_desc_dict3)
floor_desc_dict.update(floor_desc_dict4)
epc_train['floor_description'] = epc_train['FLOOR_DESCRIPTION'].replace(floor_desc_dict)
epc_test['floor_description'] = epc_test['FLOOR_DESCRIPTION'].replace(floor_desc_dict)

Tidying up levels not included in the chaid

In [128]:
floor_desc_dict_extra = dict.fromkeys(['average thermal transmittance 1.6 w/m²k', 'average thermal transmittance 2.4 w/m²k', 'average thermal transmittance 1.9 w/m²k','trawsyriannedd thermol cyfartalog 0.15 w/m²k','suspended'],'I dont know')
floor_desc_dict_extra1 = dict.fromkeys(['average thermal transmittance 2.4 w/m²k', 'average thermal transmittance 2.5 w/m²k'],'Insulated Timber 1990 above')
floor_desc_dict_extra2 = dict.fromkeys(['average thermal transmittance 1.8 w/m²k','solet, inswleiddio cyfyngedig (rhagdybiaeth)', 'average thermal transmittance 3.1 w/m²k'],'Concrete uninsulated floor pre 1990')
floor_desc_dict_extra.update(floor_desc_dict_extra1)
floor_desc_dict_extra.update(floor_desc_dict_extra2)
epc_train['floor_description'] = epc_train['floor_description'].replace(floor_desc_dict_extra)
epc_test['floor_description'] = epc_test['floor_description'].replace(floor_desc_dict_extra)

### Floor Level

In [129]:
# floor_level_dict = dict.fromkeys(['Ground','ground floor','Basement',],'1')
# floor_level_dict1 = dict.fromkeys(['1st','2nd','3rd','4th'],'2')
# floor_level_dict2 = dict.fromkeys(['mid floor','5th','6th','7th','8th','9th','10th','11th'],'3')
# floor_level_dict3 = dict.fromkeys(['top floor','12th','13th','14th','15th','16th','17th','18th','19th','20th',
#                                    '21st or above'],'4+')
# floor_level_dict.update(floor_level_dict1)
# floor_level_dict.update(floor_level_dict2)
# floor_level_dict.update(floor_level_dict3)
# epc_train['floor_level'] = epc_train['FLOOR_LEVEL'].replace(floor_level_dict)
# epc_test['floor_level'] = epc_test['FLOOR_LEVEL'].replace(floor_level_dict)

### Glazed Type

In [130]:
glazed_dict = dict.fromkeys(['double glazing installed before 2002','double glazing, unknown install date'],'Double glazed')
glazed_dict1 = dict.fromkeys(['triple, known data','triple glazing'],'Double glazed from 2020')
glazed_dict2 = dict.fromkeys(['secondary glazing','not defined','single glazing'],'I dont know')
glazed_dict3 = dict.fromkeys(['double, known data','double glazing installed during or after 2002'],'Single glazed')
glazed_dict.update(glazed_dict1)
glazed_dict.update(glazed_dict2)
glazed_dict.update(glazed_dict3)
# glazed_dict['INVALID!'] = np.nan
epc_train['glazed_type'] = epc_train['GLAZED_TYPE'].replace(glazed_dict)
epc_test['glazed_type'] = epc_test['GLAZED_TYPE'].replace(glazed_dict)

### Hot water description

In [131]:
hot_water_dict = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node1'],'Gas Boiler')
hot_water_dict1 = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node2'],'Oil Boiler')
hot_water_dict2 = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node3'],'Electric Heating')
hot_water_dict3 = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node4'],'Solar')
hot_water_dict4 = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node5'],'Fuel Boiler')
hot_water_dict5 = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node6'],'Coal Boiler')
hot_water_dict.update(hot_water_dict1)
hot_water_dict.update(hot_water_dict2)
hot_water_dict.update(hot_water_dict3)
hot_water_dict.update(hot_water_dict4)
hot_water_dict.update(hot_water_dict5)
epc_train['hotwater_description'] = epc_train['HOTWATER_DESCRIPTION'].replace(hot_water_dict)
epc_test['hotwater_description'] = epc_test['HOTWATER_DESCRIPTION'].replace(hot_water_dict)

In [132]:
# hot_water_dict = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node1'],'water group 1')
# hot_water_dict1 = dict.fromkeys(chaid_dict['HOTWATER_DESCRIPTION']['node2'],'water group 2')
# hot_water_dict.update(hot_water_dict1)
# epc_train['hotwater_description'] = epc_train['HOTWATER_DESCRIPTION'].replace(hot_water_dict)
# epc_test['hotwater_description'] = epc_test['HOTWATER_DESCRIPTION'].replace(hot_water_dict)

In [133]:
# water_dict_extra = dict.fromkeys(['7-hour tariff (on-peak)',
#                                   'From community scheme, plus solar',
#                                   'From main system, 7-hour tariff (on-peak)',
#                                   'Solid fuel range cooker, plus solar, no cylinder thermostat',
#                                   'Gas boiler/circulator, plus solar, no cylinder thermostat',
#                                   'Electric immersion, standard tariff, plus solar, waste water heat recovery',
#                                   'Solid fuel boiler/circulator, plus solar','Community scheme with CHP',
#                                   'Oil boiler/circulator, waste water heat recovery',
#                                   'From secondary system, no cylinder thermostat, plus solar',
#                                   'From secondary system, no cylinderstat, plus solar',
#                                   'From second main heating system, plus solar', 'Gas range cooker, plus solar',
#                                   'Gas range cooker, plus solar, no cylinder thermostat',
#                                   'Heat pump, waste water heat recovery','SAP:Hot-Water',
#                                   'From main system, no cylinder thermostat, flue gas heat recovery'
#                                  ],'water group 1')
# water_dict_extra1 = dict.fromkeys(['O system eilaidd','Room heaters, anthracite',
#                                    'Oil range cooker, plus solar',
#                                    'Oil range cooker, plus solar, no cylinder thermostat',
#                                    'From main system, no cylinderstat, no cylinderstat',
#                                    'No system present : electric immersion assumed', 'Electric immersion',
#                                    'From secondary heater, standard tariff','From secondary heater',
#                                    'No system present?electric immersion assumed', ', no cylinderstat',
#                                    'Single-point gas water heater', 'Point gas water heater, no cylinderstat',
#                                    'Back boiler (hot water only), gas','Gas multipoint, no cylinder thermostat',
#                                    'Electric instantaneous at point of use, no cylinder thermostat',
#                                    'No hot water system present - electric immersion assumed, plus solar',
#                                    'No system present: electric immersion assumed, no cylinder thermostat',
#                                    'Solid fuel range cooker, no cylinderstat','Gas boiler/circulator, no cylinderstat'
#                                  ],'water group 2')
# water_dict_extra.update(water_dict_extra1)
# epc_train['hotwater_description'] = epc_train['hotwater_description'].replace(water_dict_extra)
# epc_test['hotwater_description'] = epc_test['hotwater_description'].replace(water_dict_extra)

In [134]:
water_dict_extra = dict.fromkeys(['Back boiler (hot water only), gas', 'Community scheme with CHP, no cylinderstat', 'Electric immersion (on-peak or off-peak)',
                                    'Gas boiler/circulator for water heating only', 'Gas instantaneous at point of use, plus solar'], 'Gas Boiler') 
water_dict_extra1 = dict.fromkeys(['Electric immersion, standard tariff, no cylinderstat, plus solar', 'From main system, flue gas heat recovery, plus solar, waste water heat recovery'], 'Electric Heating')
water_dict_extra2 = dict.fromkeys(['From secondary system, no cylinderstat, plus solar','From secondary system, standard tariff', 'Solid fuel boiler/circulator for water heating only, plus solar'], 'Solar')
water_dict_extra.update(water_dict_extra1)
water_dict_extra.update(water_dict_extra2)
epc_train['hotwater_description'] = epc_train['hotwater_description'].replace(water_dict_extra)
epc_test['hotwater_description'] = epc_test['hotwater_description'].replace(water_dict_extra)

In [135]:
# putting remaining small options into one group, note leaving 'From main system' separate as it's 75% of all values
# hotwater_leftover = [x for x in set(epc_train['hotwater_description']) if 'water group' not in str(x)]
# hotwater_leftover_test = [x for x in set(epc_test['hotwater_description']) if 'water group' not in str(x)]
#hotwater_leftover.remove('From main system')
#hotwater_leftover_test.remove('From main system')
# hotwater_leftover.extend(hotwater_leftover_test)
# hotwater_leftover_dict = dict.fromkeys(hotwater_leftover,'Coal boiler')
# epc_train['hotwater_description'] = epc_train['hotwater_description'].replace(hotwater_leftover_dict)
# epc_test['hotwater_description'] = epc_test['hotwater_description'].replace(hotwater_leftover_dict)

###  Lighting Description

In [136]:
# epc_train['low_energy_lighting_perc'] = epc_train.apply(lambda row: 100.0 if row['LIGHTING_DESCRIPTION'] == 'low energy lighting in all fixed outlets' else row['low_energy_lighting_perc'], axis=1)
# epc_train['low_energy_lighting_perc'] = epc_train.apply(lambda row: 0 if row['LIGHTING_DESCRIPTION'] == 'no low energy lighting' else row['low_energy_lighting_perc'], axis=1)
# epc_test['low_energy_lighting_perc'] = epc_test.apply(lambda row: 100.0 if row['LIGHTING_DESCRIPTION'] == 'low energy lighting in all fixed outlets' else row['low_energy_lighting_perc'], axis=1)
# epc_test['low_energy_lighting_perc'] = epc_test.apply(lambda row: 0 if row['LIGHTING_DESCRIPTION'] == 'no low energy lighting' else row['low_energy_lighting_perc'], axis=1)

In [137]:
chaid_dict['LIGHTING_DESCRIPTION']['node1'].remove('<missing>')

In [138]:
# lighting_dict = dict.fromkeys(chaid_dict['LIGHTING_DESCRIPTION']['node1'],'Low energy lighting')
# lighting_dict1 = dict.fromkeys(chaid_dict['LIGHTING_DESCRIPTION']['node2'],'I dont know')
# lighting_dict2 = dict.fromkeys(chaid_dict['LIGHTING_DESCRIPTION']['node3'],'No low energy lighting')
# lighting_dict.update(lighting_dict1)
# lighting_dict.update(lighting_dict2)
# epc_train['lighting_description'] = epc_train['LIGHTING_DESCRIPTION'].replace(lighting_dict)
# epc_test['lighting_description'] = epc_test['LIGHTING_DESCRIPTION'].replace(lighting_dict)

In [139]:
lighting_dict = dict.fromkeys(['low energy lighting 70% of fixed outlets','low energy lighting 90% of fixed outlets',
                                     'low energy lighting 100% of fixed outlets', 'low energy lighting 80% of fixed outlets',
                                     'no low energy lighting'],'No low energy lighting')
lighting_dict1 = dict.fromkeys(['low energy lighting 20% of fixed outlets', 'low energy lighting 30% of fixed outlets',
                                     'low energy lighting 40% of fixed outlets','low energy lighting 50% of fixed outlets',
                                     'low energy lighting 60% of fixed outlets', 'low energy lighting in all fixed outlets'], 'Low energy lighting')
lighting_dict2 = dict.fromkeys(['goleuadau ynni-isel ym mhob un ogçör mannau gosod', 'low energy lighting 0% of fixed outlets',
                                     'low energy lighting 10% of fixed outlets'], 'I dont know')
lighting_dict.update(lighting_dict1)
lighting_dict.update(lighting_dict2)
epc_train['lighting_description'] = epc_train['LIGHTING_DESCRIPTION'].replace(lighting_dict)
epc_test['lighting_description'] = epc_test['LIGHTING_DESCRIPTION'].replace(lighting_dict)

### Main heating controls

In [140]:
chaid_dict['MAINHEATCONT_DESCRIPTION']['node1'].remove('<missing>')

In [141]:
main_heat_dict = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node1'],'Programmer: time control')
main_heat_dict1 = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node2'],'Programmer: room thermostat')
main_heat_dict2 = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node3'],'Programmer: no room thermostat')
main_heat_dict3 = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node4'],'Programmer: TRVs')
main_heat_dict4 = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node5'],'Community heating charging system')
main_heat_dict5 = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node6'],'I dont know')
main_heat_dict6 = dict.fromkeys(chaid_dict['MAINHEATCONT_DESCRIPTION']['node7'],'Room thermostat')
main_heat_dict.update(main_heat_dict1)
main_heat_dict.update(main_heat_dict2)
main_heat_dict.update(main_heat_dict3)
main_heat_dict.update(main_heat_dict4)
main_heat_dict.update(main_heat_dict5)
main_heat_dict.update(main_heat_dict6)
epc_train['mainheat_controls'] = epc_train['MAINHEATCONT_DESCRIPTION'].replace(main_heat_dict)
epc_test['mainheat_controls'] = epc_test['MAINHEATCONT_DESCRIPTION'].replace(main_heat_dict)

In [142]:
main_heat_dict_extra = dict.fromkeys(['Charging system linked to use of communit heating, TRVs', 'Time and temperature zone control by suitable arrangement of plumbing and electrical services',
                                      'Dim rheolaeth amser na rheolaeth thermostatig ar dymheredd yr ystafell'],'Community heating charging system')
main_heat_dict_extra1 = dict.fromkeys(["Thermostat ystafell yn unig", 'Dim rheolaeth thermostatig ar dymheredd yr ystafell',
                                        'Manual charge control'],'Room thermostat')
main_heat_dict_extra2 = dict.fromkeys(["Rhaglennydd ac o leiaf ddau thermostat ystafell",
                                       'Rheoli gwefr drydanol yn awtomatig',
                                       'Programmer + appliance thermostats',
                                       'Rhaglennydd a thermostatau ar y cyfarpar',
                                       'Programmer + TRVs + boiler energy manager',
                                       'Programmer + TRVs + flow switch',
                                       'Programmer + room thermostats',
                                       'No thermostatic control of room temperature'
                                      ],'Prgrammer: TRVs')
main_heat_dict_extra3 = dict.fromkeys(['Charging system linked to use of communit heating, programmer and TRVs',
                                       'Programmer and delayed start thermostat','Appliance thermostat','CELECT-type controls'
                                      ],'Prgrammer: room thermostat')
main_heat_dict_extra4 = dict.fromkeys(['No time or thermostatic control of room temp','No time or thermostatic control of room temperature',
                                        'None', 'Not relevant (supplies DHW only)','Rheoli\'r tâl â llaw','RheoliGÃ‡Ã-r t+Ã³l +Ã³ llaw'
                                      ],'I dont know')
main_heat_dict_extra5 = dict.fromkeys(['Programmer, no room thermostat', 'Programmer, no thermostat', 'Rhaglennydd a thermostat ystafell',
                                        'Rhaglennydd, TRVs a falf osgoi','Rheoli?r tal a llaw','RheoliGÇÖr t+ól +ó llaw'],'Programmer: no room thermostat')
main_heat_dict_extra.update(main_heat_dict_extra1)
main_heat_dict_extra.update(main_heat_dict_extra2)
main_heat_dict_extra.update(main_heat_dict_extra3)
epc_train['mainheat_controls'] = epc_train['mainheat_controls'].replace(main_heat_dict_extra)
epc_test['mainheat_controls'] = epc_test['mainheat_controls'].replace(main_heat_dict_extra)

### Property type

In [143]:
# prop_type_dict = dict.fromkeys(['Bungalow','Park home'],'one storey building')
# epc_train['property_type'] = epc_train['PROPERTY_TYPE'].replace(prop_type_dict)
# epc_test['property_type'] = epc_test['PROPERTY_TYPE'].replace(prop_type_dict)

In [144]:
prop_type_dict = dict.fromkeys(chaid_dict['PROPERTY_TYPE']['node1'],'One storey building')
prop_type_dict1 = dict.fromkeys(chaid_dict['PROPERTY_TYPE']['node2'],'Multiple storey building')
prop_type_dict2 = dict.fromkeys(chaid_dict['PROPERTY_TYPE']['node3'],'Few storey building')
prop_type_dict.update(prop_type_dict1)
prop_type_dict.update(prop_type_dict2)
epc_train['property_type'] = epc_train['PROPERTY_TYPE'].replace(prop_type_dict)
epc_test['property_type'] = epc_test['PROPERTY_TYPE'].replace(prop_type_dict)

### Roof description

In [145]:
chaid_dict['ROOF_DESCRIPTION']['node1'].remove('<missing>')

In [146]:
roof_dict = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node1'],'I dont know')
roof_dict1 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node2'],'Insulated horizontally from 2021')
roof_dict2 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node3'],'Insulated horizontally from 2002 to 2021')
roof_dict3 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node4'],'Insulated horizontally from 1990 to 2002')
roof_dict4 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node5'],'Insulated on the slope from 2021')
roof_dict5 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node6'],'Insulated on the slope from 2002 To 2021')
roof_dict6 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node7'],'Insulated on the slope from 1990 To 2001')
roof_dict7 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node8'],'Central building Regulations: new build')
roof_dict8 = dict.fromkeys(chaid_dict['ROOF_DESCRIPTION']['node9'],'No insulation')
roof_dict.update(roof_dict1)
roof_dict.update(roof_dict2)
roof_dict.update(roof_dict3)
roof_dict.update(roof_dict4)
roof_dict.update(roof_dict5)
roof_dict.update(roof_dict6)
roof_dict.update(roof_dict7)
roof_dict.update(roof_dict8)

epc_train['roof_description'] = epc_train['ROOF_DESCRIPTION'].replace(roof_dict)
epc_test['roof_description'] = epc_test['ROOF_DESCRIPTION'].replace(roof_dict)

In [147]:
roof_dict_extra = dict.fromkeys(['average thermal transmittance 1.7 w/m²k', 'average thermal transmittance 3.1 w/m²k'], 'Insulated on the slope from 1990 To 2001') 
roof_dict_extra1 = dict.fromkeys(['average thermal transmittance 4.8 w/m²k', 'Pitched, insulated at raftersu', 'average thermal transmittance 4.0 w/m²k'], 'No insulation')
roof_dict_extra.update(roof_dict_extra1)
epc_train['roof_description'] = epc_train['roof_description'].replace(roof_dict_extra)
epc_test['roof_description'] = epc_test['roof_description'].replace(roof_dict_extra)

In [148]:
# putting remaining small options into one group, note leaving 'From main system' separate as it's 75% of all values
# roof_leftover = [x for x in set(epc_train['roof_description']) if 'roof group' not in str(x)]
# roof_leftover_test = [x for x in set(epc_test['roof_description']) if 'roof group' not in str(x)]
# roof_leftover.extend(roof_leftover_test)
# roof_leftover_dict = dict.fromkeys(roof_leftover,'I dont know')
# epc_train['roof_description'] = epc_train['roof_description'].replace(roof_leftover_dict)
# epc_test['roof_description'] = epc_test['roof_description'].replace(roof_leftover_dict)

### Transaction type

In [149]:
# chaid_dict['TRANSACTION_TYPE']['node1'].remove('<missing>')

In [150]:
# trans_dict = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node1'],'private rental and sale')
# trans_dict1 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node2'],'social rental and new build')
# trans_dict2 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node3'],'private rental and sale')
# trans_dict3 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node4'],'social rental and new build')
# #trans_dict4 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node5'],'assessment')
# trans_dict.update(trans_dict1)
# trans_dict.update(trans_dict2)
# trans_dict.update(trans_dict3)
# #trans_dict.update(trans_dict4)
# trans_dict['unknown'] = np.nan
# epc_train['transaction_type'] = epc_train['TRANSACTION_TYPE'].replace(trans_dict)
# epc_test['transaction_type'] = epc_test['TRANSACTION_TYPE'].replace(trans_dict)


In [151]:
trans_dict = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node1'],'Private rental')
trans_dict1 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node2'],'Sale')
trans_dict2 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node3'],'New dwelling')
trans_dict3 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node4'],'Stock condition survey')
trans_dict4 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node5'],'Green deal assessment')
trans_dict5 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node6'],'Social rental')
trans_dict6 = dict.fromkeys(chaid_dict['TRANSACTION_TYPE']['node7'],'I dont know')
trans_dict.update(trans_dict1)
trans_dict.update(trans_dict2)
trans_dict.update(trans_dict3)
trans_dict.update(trans_dict4)
trans_dict.update(trans_dict5)
trans_dict.update(trans_dict6)
# trans_dict['unknown'] = np.nan
epc_train['transaction_type'] = epc_train['TRANSACTION_TYPE'].replace(trans_dict)
epc_test['transaction_type'] = epc_test['TRANSACTION_TYPE'].replace(trans_dict)

### Walls description

In [152]:
chaid_dict['WALLS_DESCRIPTION']['node1'].remove('<missing>')

In [153]:
walls_dict = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node1'],'I dont know')
walls_dict1 = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node2'],'Cavity wall from 2021 to present')
walls_dict2 = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node3'],'Cavity wall 2002 - 2020')
walls_dict3 = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node4'],'Cavity wall 1980 - 2001')
walls_dict4 = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node5'],'Cavity wall, insulated')
walls_dict5 = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node6'],'Cavity wall 1960 - 1980')
walls_dict6 = dict.fromkeys(chaid_dict['WALLS_DESCRIPTION']['node7'],'No insulation')
walls_dict.update(walls_dict1)
walls_dict.update(walls_dict2)
walls_dict.update(walls_dict3)
walls_dict.update(walls_dict4)
walls_dict.update(walls_dict5)
walls_dict.update(walls_dict6)
epc_train['walls_description'] = epc_train['WALLS_DESCRIPTION'].replace(walls_dict)
epc_test['walls_description'] = epc_test['WALLS_DESCRIPTION'].replace(walls_dict)

In [155]:
wall_dict_extra = dict.fromkeys(['average thermal transmittance 2.9 w/m²k', 'average thermal transmittance 3.3 w/m²k', 'granite or whinstone, filled cavity', 'average thermal transmittance 3.5 w/m²k'], 'Cavity wall 1980 - 2001') 
wall_dict_extra1 = dict.fromkeys(['sandstone,', 'timber frame, filled cavity and external insulation', 'timber frame, filled cavity and internal insulation', 'average thermal transmittance 3.1 w/m²k'], 'Cavity wall, insulated')
wall_dict_extra.update(wall_dict_extra1)
epc_train['walls_description'] = epc_train['walls_description'].replace(wall_dict_extra)
epc_test['walls_description'] = epc_test['walls_description'].replace(wall_dict_extra)

In [156]:
# putting remaining small options into one group, note leaving 'From main system' separate as it's 75% of all values
# walls_leftover = [x for x in set(epc_train['walls_description']) if 'walls group' not in str(x)]
# walls_leftover_test = [x for x in set(epc_test['walls_description']) if 'walls group' not in str(x)]
# walls_leftover.extend(walls_leftover_test)
# walls_leftover_dict = dict.fromkeys(walls_leftover,'I dont know')
# epc_train['walls_description'] = epc_train['walls_description'].replace(walls_leftover_dict)
# epc_test['walls_description'] = epc_test['walls_description'].replace(walls_leftover_dict)

### Windows Description

In [157]:
windows_dict = dict.fromkeys(['single glazing and secondary glazing', 'single glazing|single glazing', 'single glazing',
                                    'single and multiple glazing', 'single glazing and double glazing', 'partial multiple glazingdouble glazing',
                                    'partial single glazing','some multiple glazing'],'Single glazed')
# windows_dict1 = dict.fromkeys(['single and multiple glazing', 'single glazing and double glazing'],'Single glazing: Metal frame')
windows_dict1 = dict.fromkeys(['partial secondary glazing','mostly multiple glazing','partial double glazing',
                             'secondary glazing', 'double glazing', 'partial double glazing|partial double glazing',
                             'some secondary glazing','some double glazing',
                             'mostly secondary glazing', 'mostly double glazing', 'partial multiple glazing',
                             'full secondary glazing', 'some multiple glazing''full double glazing',
                             'full double glazing|full double glazing', 'full double glazing', 'partial double glazingdouble glazing'],'Double glazed')
# windows_dict3 = dict.fromkeys(['mostly secondary glazing', 'mostly double glazing', 'partial multiple glazing',
#                              'full secondary glazing', 'some multiple glazing''full double glazing',
#                              'full double glazing|full double glazing'],'Standard double glazing: Metal frame')
windows_dict2 = dict.fromkeys(['some triple glazing', 'partial triple glazing', 'high performance glazing',
                             'high performance glazing','mostly triple glazing', 'multiple glazing throughout', 'full triple glazing',
                             'high performance glazing|high performance glazing','full double glazingdouble glazing',
                                'multiple glazing throughout double glazing','multiple glazing throughout|multiple glazing throughout'
                                ], 'Double glazed from 2020')
# windows_dict5 = dict.fromkeys(['mostly triple glazing', 'multiple glazing throughout', 'full triple glazing',
#                              'high performance glazing|high performance glazing'],'Triple glazing: Metal frame')
windows_dict3 = dict.fromkeys(['solid, no insulation (assumed)', 'suspended, no insulation (assumed)', 'unknown complex glazing regime',
                             'gwydrau triphlyg gan mwyaf'],'I dont know')
windows_dict.update(windows_dict1)
windows_dict.update(windows_dict2)
windows_dict.update(windows_dict3)
# windows_dict.update(windows_dict4)
# windows_dict.update(windows_dict5)
# windows_dict.update(windows_dict6)
epc_train['window_description'] = epc_train['WINDOWS_DESCRIPTION'].replace(windows_dict)
epc_test['window_description'] = epc_test['WINDOWS_DESCRIPTION'].replace(windows_dict)

In [158]:
# windows_dict = dict.fromkeys(chaid_dict['WINDOWS_DESCRIPTION']['node1'],'window group 1')
# windows_dict1 = dict.fromkeys(chaid_dict['WINDOWS_DESCRIPTION']['node2'],'window group 2')
# windows_dict2 = dict.fromkeys(chaid_dict['WINDOWS_DESCRIPTION']['node3'],'window group 3')
# windows_dict.update(windows_dict1)
# windows_dict.update(windows_dict2)
# epc_train['window_description'] = epc_train['WINDOWS_DESCRIPTION'].replace(windows_dict)
# epc_test['window_description'] = epc_test['WINDOWS_DESCRIPTION'].replace(windows_dict)

In [159]:
# epc_train['window_description'] = epc_train['window_description'].replace('multiple glazing throughout double glazing','window group 3')
# epc_test['window_description'] = epc_test['window_description'].replace('multiple glazing throughout double glazing','window group 3')

### Region

In [160]:
# region_dict = dict.fromkeys(['Blaenau Gwent','Neath Port Talbot','Pembrokeshire','Rhondda Cynon Taf','Caerphilly',
#                              'Flintshire','Carmarthenshire','Powys','Conwy','Ceredigion','Debighshire',
#                              'Gwynedd','Isle of Anglesey'],'rural')
# region_dict1 = dict.fromkeys(['Bridgend','Monmouthshire','Wrexham','Merthyr Tydfil','Vale of Glamorgan','Cardiff',
#                               'Torfaen','Newport','Swansea'],'suburban')
# region_dict.update(region_dict1)
# epc_train['locality'] = epc_train['region'].replace(region_dict)
# epc_test['locality'] = epc_test['region'].replace(region_dict)

## Binning Numeric Fields

In [161]:
def numberic_bins(var,bin_boundaries,bin_labels):

  var_new = pd.cut(var,bins = bin_boundaries,labels = bin_labels)

  return var_new

### Extension count

In [162]:
extension_max = epc_train['EXTENSION_COUNT'].max()
extension_bins = [-1,0,1,extension_max]
extension_labels = ['0','1','2+']

In [163]:
epc_train['extension'] = numberic_bins(epc_train['EXTENSION_COUNT'],extension_bins,extension_labels)
epc_test['extension'] = numberic_bins(epc_test['EXTENSION_COUNT'],extension_bins,extension_labels)

### Floor height

In [164]:
floor_max = epc_train['FLOOR_HEIGHT'].max()
floor_bins = [0,2.3,2.4,2.4999,2.5,2.7,floor_max]
floor_labels = ['0-2.3','2.3-2.4','2.4-2.5','2.5','2.5-2.7','2.7+']

In [165]:
epc_train['floor_height'] = numberic_bins(epc_train['FLOOR_HEIGHT'],floor_bins,floor_labels)
epc_test['floor_height'] = numberic_bins(epc_test['FLOOR_HEIGHT'],floor_bins,floor_labels)

### Number of habitable rooms

In [166]:
room_max = epc_train['NUMBER_HABITABLE_ROOMS'].max()
room_bins = [0,1,2,3,4,5,room_max]
room_labels = ['1','2','3','4','5','6+']

In [167]:
epc_train['habitable_rooms'] = numberic_bins(epc_train['NUMBER_HABITABLE_ROOMS'],room_bins,room_labels)
epc_test['habitable_rooms'] = numberic_bins(epc_test['NUMBER_HABITABLE_ROOMS'],room_bins,room_labels)

### Number of open fireplaces

In [168]:
fire_max = epc_train['NUMBER_OPEN_FIREPLACES'].max()
fire_bins = [-1,0,1,fire_max]
fire_labels = ['0','1','2+']

In [169]:
epc_train['open_fireplaces'] = numberic_bins(epc_train['NUMBER_OPEN_FIREPLACES'],fire_bins,fire_labels)
epc_test['open_fireplaces'] = numberic_bins(epc_test['NUMBER_OPEN_FIREPLACES'],fire_bins,fire_labels)

In [170]:
epc_train.head()

,LMK_KEY,region,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,COUNTY,...,mainheat_controls,property_type,roof_description,transaction_type,walls_description,window_description,extension,floor_height,habitable_rooms,open_fireplaces
0,472d64f27beec445462f8b8f4c9389b9ab78a69d00eb16...,Lichfield,WS14 9RF,10002368118,C,69,House,Detached,2022-01-10,Staffordshire,...,I dont know,Few storey building,Insulated horizontally from 1990 to 2002,Stock condition survey,Cavity wall from 2021 to present,Double glazed,0,2.3-2.4,6+,0
1,917209354732013042122465136978600,Craven,LA2 7JX,5006647078,D,61,House,End-Terrace,2013-04-19,North Yorkshire,...,I dont know,Few storey building,Insulated horizontally from 1990 to 2002,Stock condition survey,Cavity wall 1980 - 2001,Double glazed,0,NaN,4,0
2,524323979942010080813071072800868,Eden,CA11 0LR,8923278768,F,29,House,Detached,2010-08-06,Cumbria,...,Programmer: room thermostat,Few storey building,Insulated horizontally from 1990 to 2002,Stock condition survey,Cavity wall 1980 - 2001,Double glazed,1,2.7+,6+,2+
3,1171297749962014071015572165608334,Braintree,CO9 4QN,3669445278,D,55,House,Detached,2014-07-10,Essex,...,I dont know,Few storey building,Central building Regulations: new build,New dwelling,Cavity wall 2002 - 2020,Double glazed,1,NaN,6+,0
4,5b560fa64623f39f6bbf19ce8d65fd500fc71e539af00d...,Teignbridge,TQ12 5FE,10002120353,C,77,House,End-Terrace,2022-02-03,Devon,...,Programmer: room thermostat,Few storey building,Insulated horizontally from 1990 to 2002,Stock condition survey,Cavity wall from 2021 to present,Double glazed,0,2.3-2.4,5,0


In [171]:
epc_test.head()

,LMK_KEY,region,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,COUNTY,...,mainheat_controls,property_type,roof_description,transaction_type,walls_description,window_description,extension,floor_height,habitable_rooms,open_fireplaces
0,1575003979922017091413003983878383,Tendring,CO7 7NS,8803793578,C,71,Flat,Detached,2017-09-13,Essex,...,I dont know,Multiple storey building,Insulated on the slope from 1990 To 2001,I dont know,Cavity wall 2002 - 2020,Double glazed,0,NaN,3,0
1,923396987952013050120194798070405,Woking,GU22 9PY,4596787078,D,65,House,End-Terrace,2013-05-01,Surrey,...,I dont know,Few storey building,Insulated horizontally from 1990 to 2002,Stock condition survey,Cavity wall 2002 - 2020,Double glazed,0,NaN,3,0
2,816081039802013062010474405072108,Brentwood,CM14 4RL,1880230078,B,81,Flat,Enclosed Mid-Terrace,2013-06-20,Essex,...,I dont know,Multiple storey building,Insulated horizontally from 2002 to 2021,Social rental,Cavity wall from 2021 to present,Double glazed from 2020,NaN,NaN,NaN,0
3,199888290962008121710161105968528,Oxford,OX4 2ET,2093975568,D,57,House,End-Terrace,2008-12-16,Oxfordshire,...,I dont know,Few storey building,Insulated on the slope from 2021,Private rental,Cavity wall 1960 - 1980,Double glazed,2+,2.3-2.4,5,0
4,1412714249942016021210042743269998,Dover,CT3 3DL,1981942478,D,64,House,Mid-Terrace,2016-02-11,Kent,...,Programmer: room thermostat,Few storey building,Insulated on the slope from 2002 To 2021,New dwelling,Cavity wall 2002 - 2020,Double glazed,0,NaN,5,0


## Dropping fields

In [172]:
# features that correlate with each other or are leading
correlated_variables = ['CO2_EMISS_CURR_PER_FLOOR_AREA','CO2_EMISSIONS_CURRENT','ENERGY_CONSUMPTION_CURRENT',
                     'HEATING_COST_CURRENT','HOT_WATER_COST_CURRENT','HOT_WATER_ENERGY_EFF','HOT_WATER_ENV_EFF',
                     'LIGHTING_COST_CURRENT','LIGHTING_ENERGY_EFF','LIGHTING_ENV_EFF','LMK_KEY','LOW_ENERGY_LIGHTING',
                     'MAIN_FUEL','MAINHEAT_ENERGY_EFF','MAINHEAT_ENV_EFF','MAINHEATC_ENERGY_EFF','MAINHEATC_ENV_EFF',
                     'MAINHEATCONT_DESCRIPTION','MECHANICAL_VENTILATION','MULTI_GLAZE_PROPORTION','NUMBER_HEATED_ROOMS',
                     'POSTCODE','ROOF_ENERGY_EFF','ROOF_ENV_EFF','SECONDHEAT_DESCRIPTION','WALLS_ENERGY_EFF',
                     'WALLS_ENV_EFF','WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF']

# features replace with binned features
replace_features = ['CURRENT_ENERGY_RATING','PROPERTY_TYPE','BUILT_FORM','INSPECTION_DATE','TRANSACTION_TYPE',
                   'ENERGY_TARIFF','FLOOR_LEVEL','GLAZED_TYPE','EXTENSION_COUNT','NUMBER_HABITABLE_ROOMS',
                    'NUMBER_OPEN_FIREPLACES','HOTWATER_DESCRIPTION','FLOOR_DESCRIPTION','MAIN_HEATING_CONTROLS',
                    'WINDOWS_DESCRIPTION','WALLS_DESCRIPTION','ROOF_DESCRIPTION','LIGHTING_DESCRIPTION',
                    'FLOOR_HEIGHT']

# other fields not needed
fields_to_drop = ['region','floors_average_thermal_transmittance','low_energy_lighting_perc',
                  'roof_average_thermal_transmittance','walls_average_thermal_transmittance',
                  'COUNTY','BUILDING_REFERENCE_NUMBER','MAINS_GAS_FLAG','HEAT_LOSS_CORRIDOR'] 

# 'floor_level', 'inspection_year'

In [173]:
epc_train.drop(correlated_variables,axis = 1,inplace=True)
epc_train.drop(replace_features,axis = 1,inplace=True)
epc_train.drop(fields_to_drop,axis = 1,inplace=True)
epc_test.drop(correlated_variables,axis = 1,inplace=True)
epc_test.drop(replace_features,axis = 1,inplace=True)
epc_test.drop(fields_to_drop,axis = 1,inplace=True)

In [174]:
epc_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3867189 entries, 0 to 3867188
Data columns (total 19 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   CURRENT_ENERGY_EFFICIENCY  int64   
 1   TOTAL_FLOOR_AREA           float64 
 2   inspection_year            int64   
 3   built_form                 object  
 4   energy_tariff              object  
 5   floor_description          object  
 6   glazed_type                object  
 7   hotwater_description       object  
 8   lighting_description       object  
 9   mainheat_controls          object  
 10  property_type              object  
 11  roof_description           object  
 12  transaction_type           object  
 13  walls_description          object  
 14  window_description         object  
 15  extension                  category
 16  floor_height               category
 17  habitable_rooms            category
 18  open_fireplaces            category
dtypes: category(4), float

In [175]:
epc_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966798 entries, 0 to 966797
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   CURRENT_ENERGY_EFFICIENCY  966798 non-null  int64   
 1   TOTAL_FLOOR_AREA           966798 non-null  float64 
 2   built_form                 948263 non-null  object  
 3   energy_tariff              878570 non-null  object  
 4   floor_description          965374 non-null  object  
 5   glazed_type                799450 non-null  object  
 6   hotwater_description       964188 non-null  object  
 7   lighting_description       965405 non-null  object  
 8   mainheat_controls          964183 non-null  object  
 9   property_type              966798 non-null  object  
 10  roof_description           965252 non-null  object  
 11  transaction_type           966035 non-null  object  
 12  walls_description          965392 non-null  object  
 13  window_descrip

## Exporting data

In [177]:
epc_train.to_csv(os.path.join(processing_path,epc_train_data_fname) + epc_fname_suffix,index = False)
epc_test.to_csv(os.path.join(processing_path,epc_test_data_fname) + epc_fname_suffix,index = False)